In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import numpy as np
import pandas as pd
import os

# 파장 정의
WAVELENGTHS = np.arange(380, 781, 1)
NUM_WAVELENGTHS = len(WAVELENGTHS) #401

# GitHub raw 주소 (파일들이 위치한 경로)
base_url = "https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/"

# 사용할 재료 목록
material_files = [
    ('SiO2', 'SiO2_380.csv'),
    ('WO3',  'WO3_380.csv'),
    ('Ag',   'Ag_380.csv')
]

material_data = {}
material_names = [info[0] for info in material_files]
print("재료 리스트:", material_names)

# 재료 인덱싱 매핑
material_names = sorted(set(material_names))
material_to_index = {name: i for i, name in enumerate(material_names)}
index_to_material = {i: name for name, i in material_to_index.items()}
print("재료 인덱스 매핑:", index_to_material)

# --- 재료 파일 불러오기 ---
for material_name, filename in material_files:
    file_url = base_url + filename
    print(f"처리 중: {filename} (materials: {material_name}) → {file_url}")
    try:
        data = pd.read_csv(file_url, header=None)
    except Exception as e:
        print(f"  🚫 오류 발생: {e}")
        continue

    if data.empty:
        print("  ⚠️ 경고: 파일이 비어있습니다.")
        continue

    wavelengths = data.iloc[:, 0].values  # 파장
    n = data.iloc[:, 1].values           # 실수부
    k = data.iloc[:, 2].values           # 허수부

    n_complex = n + 1j * k               # 복소 굴절률

    material_data[material_name] = (wavelengths, n_complex)
    print(f"로드 완료: {len(wavelengths)} points")

# --- 공기 굴절률 추가 ---
material_data['Air'] = np.ones(NUM_WAVELENGTHS, dtype=np.complex128)
print("Air 굴절률 추가 완료")


재료 리스트: ['SiO2', 'WO3', 'Ag']
재료 인덱스 매핑: {0: 'Ag', 1: 'SiO2', 2: 'WO3'}
처리 중: SiO2_380.csv (materials: SiO2) → https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/SiO2_380.csv
로드 완료: 401 points
처리 중: WO3_380.csv (materials: WO3) → https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/WO3_380.csv
로드 완료: 401 points
처리 중: Ag_380.csv (materials: Ag) → https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/Ag_380.csv
로드 완료: 401 points
Air 굴절률 추가 완료


In [3]:
# ────────────────────────────────────────────────────────────────────────────
# (1) 파장 정의 (380~780 nm, 1 nm 간격 → 총 401개 파장 지점)
# ────────────────────────────────────────────────────────────────────────────
λ_tensor_global = torch.tensor(WAVELENGTHS, dtype=torch.float64).to(device) * 1e-9  # [m 단위로 변환된 파장 텐서]

# ────────────────────────────────────────────────────────────────────────────
# (2) 층별 재료 정의 (MIM 구조 예시: 금속-절연체-금속)
# ────────────────────────────────────────────────────────────────────────────
material_sequence = ['Ag', 'WO3', 'Ag']  # 구조에 사용될 재료 순서 (입사면에서부터 아래 방향)

# material_data 에서 각 재료의 복소 굴절률 (n + ik) 값 추출 (shape: [401])
n_list_np_arrays = [material_data[mat][1] for mat in material_sequence]  # 길이 3 리스트 (각각 [401] 크기)

# numpy → torch 변환 후, complex64 형식으로 묶어서 GPU로 이동 (shape: [3, 401])
n_list_torch = torch.stack([
    torch.from_numpy(arr).to(torch.complex64) for arr in n_list_np_arrays
], dim=0).to(device)

# ────────────────────────────────────────────────────────────────────────────
# (3) 입사 매질 (n_i)와 출사 매질 (n_s) 설정
# ────────────────────────────────────────────────────────────────────────────
n_i_np = material_data['SiO2'][1]   # 입사 매질: SiO2의 복소 굴절률 (n + ik)
n_s_np = material_data['Air']       # 출사 매질: Air (n=1, k=0)

n_i_torch = torch.from_numpy(n_i_np).to(torch.complex64).to(device)  # torch 변환 후 GPU 이동
n_s_torch = torch.from_numpy(n_s_np).to(torch.complex64).to(device)

# ────────────────────────────────────────────────────────────────────────────
# (4) 두께 그리드 정의 (nm 단위, 총 조합 수 = 8 * 187 * 8 = 11968개)
# ────────────────────────────────────────────────────────────────────────────
d1_list_nm = np.arange(5, 41, 5)      # 금속층 (d1): 5 ~ 40 nm, 간격 5 nm → 총 8개
d2_list_nm = np.arange(150, 801, 5)    # 절연층 (d2): 20 ~ 950 nm, 간격 5 nm → 총 187개
d3_list_nm = np.arange(5, 41, 5)      # 금속층 (d3): 5 ~ 40 nm, 간격 5 nm → 총 8개

print(len(d1_list_nm)*len(d2_list_nm)*len(d3_list_nm))  # 총 조합 개수 출력 (8 × 187 × 8 = 11,968)

# ────────────────────────────────────────────────────────────────────────────
# (5) 샘플 저장용 리스트 초기화
# ────────────────────────────────────────────────────────────────────────────

all_d_tilde = []    # ▶ 각 샘플의 (d1, d2, d3) 두께 조합을 저장할 리스트
                    #   일반적으로 나중에 정규화된 형태로 저장되며 학습 입력으로 사용

all_T_target = []   # ▶ 각 조합에 대응하는 스펙트럼 T(λ) [shape: (401,)] 를 저장할 리스트
                    #   Forward model을 통해 계산된 정답 스펙트럼 (target)


8384


In [4]:
# ────────────────────────────────────────────────────────────────────────────
# (A-1) TMMNetwork 정의 (클래스 이름, dtype 통일, calculate() 추가)
# ────────────────────────────────────────────────────────────────────────────
class TMMNetwork(nn.Module):
    def __init__(self, n_list_input, n_i_input, n_s_input, wavelengths_m_tensor):
        super().__init__()
        dtype_complex = torch.complex128
        dtype_float = torch.float64

        n_list_real = n_list_input.real
        n_list_imag = -torch.abs(n_list_input.imag)
        self.register_buffer('n_list', torch.complex(n_list_real, n_list_imag))
        self.register_buffer('n_i', n_i_input.to(dtype_complex))
        self.register_buffer('n_s', n_s_input.to(dtype_complex))

        # 파수 k0
        k0 = 2 * torch.pi / torch.clamp(wavelengths_m_tensor, min=1e-20)
        self.register_buffer('k0', k0)  # 이제 k0_tilde 아님

        self.num_layers = self.n_list.shape[0]
        self.num_wavelengths = wavelengths_m_tensor.shape[0]

    def forward(self, thicknesses_nm):
        device = thicknesses_nm.device
        dtype_complex = torch.complex128
        imag_unit = torch.tensor(1j, dtype=dtype_complex, device=device)

        if thicknesses_nm.ndim == 1:
            thicknesses_nm = thicknesses_nm.unsqueeze(0)  # (1, 3)
        B = thicknesses_nm.shape[0]  # batch size

        # Convert to meters
        thicknesses_m = thicknesses_nm * 1e-9  # (B, 3)

        # (B, 3, 401): broadcasting layer × wavelength
        delta = thicknesses_m[:, :, None] * self.k0[None, None, :] * self.n_list[None, :, :]

        cosδ = torch.cos(delta)
        sinδ = torch.sin(delta)

        # 초기 M_total: (B, 401, 2, 2)
        M_total = torch.eye(2, dtype=dtype_complex, device=device).repeat(B, self.num_wavelengths, 1, 1)

        for j in range(self.num_layers):
            Yj = self.n_list[j] * 2.654e-3  # (401,)

            sin_j = sinδ[:, j, :]
            cos_j = cosδ[:, j, :]

            m11 = cos_j
            m12 = imag_unit * sin_j / Yj[None, :]
            m21 = imag_unit * Yj[None, :] * sin_j
            m22 = cos_j

            Mj = torch.stack([
                torch.stack([m11, m12], dim=-1),
                torch.stack([m21, m22], dim=-1)
            ], dim=-2)  # shape: (B, 401, 2, 2)

            M_total = torch.matmul(M_total, Mj)

        Y_in = self.n_i * 2.654e-3
        Y_out = self.n_s * 2.654e-3

        B_ = M_total[:, :, 0, 0] + M_total[:, :, 0, 1] * Y_out
        C_ = M_total[:, :, 1, 0] + M_total[:, :, 1, 1] * Y_out
        denom = Y_in * B_ + C_
        t1 = (2 * Y_in) / denom  # (B, 401)

        T = (torch.real(self.n_s) / torch.real(self.n_i)) * torch.abs(t1) ** 2
        return T.to(torch.float64)  # (B, 401)

model = TMMNetwork(n_list_torch, n_i_torch, n_s_torch, λ_tensor_global).to(device)

In [5]:
import os
import random
import numpy as np
import torch

# ------------------------------
# 시드 고정 (재현성 확보)
# ------------------------------
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)  # 시드 고정 실행

In [6]:
# @title
from torch.utils.data import random_split
all_d = []
all_T_target = []

print("Generating (normalized_d → T_target) pairs ...")
for d1 in d1_list_nm:
    for d2 in d2_list_nm:
        for d3 in d3_list_nm:
            # (a) [d1, d2, d3]
            d_nm_vec = torch.tensor([[d1, d2, d3]], dtype=torch.float64)  # shape: (1, 3)
            all_d.append(d_nm_vec.cpu().numpy())
            # (b) TMM forward
            T_spec = model(d_nm_vec.to(device))  # forward(nm 단위 두께)
            all_T_target.append(T_spec.detach().cpu().numpy())
# NumPy 배열로 변환
all_d = np.array(all_d, dtype=np.float64)   # shape: (3096, 3)
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ두께표준화ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
THICKNESS_MIN = torch.tensor(np.min(all_d, axis=0), dtype=torch.float64, device=device)
THICKNESS_MAX = torch.tensor(np.max(all_d, axis=0), dtype=torch.float64, device=device)
def standardize_thickness(d_nm_array):
    d_nm_array = torch.tensor(d_nm_array, dtype=torch.float64, device=THICKNESS_MIN.device) if not isinstance(d_nm_array, torch.Tensor) else d_nm_array.to(dtype=torch.float64, device=THICKNESS_MIN.device)
    return (d_nm_array - THICKNESS_MIN) / (THICKNESS_MAX - THICKNESS_MIN)

def destandardize_thickness(d_norm_array):
    d_norm_array = d_norm_array.clone().to(dtype=torch.float64, device=THICKNESS_MIN.device)
    return d_norm_array * (THICKNESS_MAX - THICKNESS_MIN) + THICKNESS_MIN

all_d_norm = standardize_thickness(all_d)  # shape: (3096, 3)
d_nm_check = destandardize_thickness(all_d_norm)


#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ스펙트럼표준화ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
all_T_target = np.array(all_T_target, dtype=np.float64) # shape: (3096, 401)
print("Total samples:", all_d.shape[0])  # 3096

SPECTRUM_MIN = torch.tensor(np.min(all_T_target, axis=0), dtype=torch.float64).to(device)
SPECTRUM_MAX = torch.tensor(np.max(all_T_target, axis=0), dtype=torch.float64).to(device)
def standardize_spectrum(T_array):
    T_tensor = torch.tensor(T_array, dtype=torch.float64).to(device)
    return (T_tensor - SPECTRUM_MIN) / (SPECTRUM_MAX - SPECTRUM_MIN)

def destandardize_spectrum(T_std_array):
    return T_std_array * (SPECTRUM_MAX - SPECTRUM_MIN) + SPECTRUM_MIN

# destandardize_spectrum의 결과(GPU 텐서)를 .cpu().numpy()로 변환하여 비교합니다.
all_T_target_std = standardize_spectrum(all_T_target)
restored_T = destandardize_spectrum(all_T_target_std)
assert np.allclose(restored_T.cpu().numpy(), all_T_target, atol=1e-5)

# ────────────────────────────────────────────────────────────────────────────
# (B) Dataset / DataLoader 구축
# ────────────────────────────────────────────────────────────────────────────
class BiTMMNormalizedDataset(Dataset):
    def __init__(self, d_norm_array, T_array):
        self.d_norm = d_norm_array.clone().to(dtype=torch.float64)
        self.T_spec = T_array.clone().detach().to(dtype=torch.float64) if isinstance(T_array, torch.Tensor) else torch.tensor(T_array, dtype=torch.float64)

    def __len__(self):
        return self.d_norm.shape[0]

    def __getitem__(self, idx):
        return {
            'd_norm': self.d_norm[idx],       # shape: (3,)1
            'T_target': self.T_spec[idx]      # shape: (401,)
        }

batch_size = 32


# 시드 고정
seed = 42
g = torch.Generator().manual_seed(seed)

# Dataset 생성
dataset = BiTMMNormalizedDataset(all_d_norm, all_T_target_std)

# 전체 길이 및 split 비율
total_size = len(dataset)
train_size = int(0.9 * total_size)
val_size   = int(0.05 * total_size)
test_size  = total_size - train_size - val_size  # 나머지

print(f"Total: {total_size}, Train: {train_size}, Val: {val_size}, Test: {test_size}")

# Dataset 분할
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size], generator=g  # g로 통일
)

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0, generator=g)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)


Generating (normalized_d → T_target) pairs ...
Total samples: 8384
Total: 8384, Train: 7545, Val: 419, Test: 420


# **Forward Training **

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

# ──────────────────────────────────────────────────────
# [1] 모델 정의: ForwardNet
# - 입력: 정규화된 두께 벡터 (3차원)
# - 출력: 정규화된 스펙트럼 벡터 (401차원)
# ──────────────────────────────────────────────────────

batch_size = 128

# MLP 각 층의 뉴런 수 (실험적으로 설정된 값)
N1 = 1190
N2 = 824
N3 = 920

class ForwardNet(nn.Module):
    def __init__(self, input_dim=3, output_dim=401):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, N1),   # 입력층 → 첫 번째 은닉층
            nn.ReLU(),
            nn.Linear(N1, N2),  # 첫 번째 → 두 번째 은닉층
            nn.ReLU(),
            nn.Linear(N2, N3),  # 두 번째 → 세 번째 은닉층
            nn.ReLU(),
            nn.Linear(N3, 401)  # 세 번째 → 출력층 (스펙트럼 401포인트)
        )

    def forward(self, x):
        return self.net(x)

# ──────────────────────────────────────────────────────
# [2] 모델 및 옵티마이저 초기화
# ──────────────────────────────────────────────────────

# 모델을 GPU + float64로 초기화
forward_net = ForwardNet(input_dim=3, output_dim=401).to(device).to(torch.float64)

# 옵티마이저: Adam, 초기 학습률 1e-3
optimizer = torch.optim.Adam(forward_net.parameters(), lr=1e-3)

# ──────────────────────────────────────────────────────
# [3] 학습 관련 파라미터 설정
# ──────────────────────────────────────────────────────
num_epochs = 600  # 전체 학습 epoch 수

# 학습 결과를 저장할 디렉토리 생성
output_dir = "C:/Users/PC/Desktop/Deep/"
os.makedirs(output_dir, exist_ok=True)

# 학습 및 검증 손실 기록용 리스트
train_loss_history = []
val_loss_history = []

# ──────────────────────────────────────────────────────
# [4] 학습 루프 시작
# ──────────────────────────────────────────────────────
for epoch in range(num_epochs):
    forward_net.train()  # 학습 모드 전환
    total_loss = 0.0     # 누적 손실 초기화

    # ▶ 한 epoch 동안 전체 train_loader 순회
    for batch in train_loader:
        d_norm = batch['d_norm'].to(device)        # 입력: 정규화된 두께 [B, 3]
        T_target = batch['T_target'].to(device)    # 정답: 정규화된 스펙트럼 [B, 401]

        optimizer.zero_grad()
        T_pred = forward_net(d_norm)               # 모델 예측

        loss = F.mse_loss(T_pred, T_target)        # 평균 제곱 오차
        loss.backward()                            # 역전파
        optimizer.step()                           # 가중치 업데이트
        total_loss += loss.item()                  # 손실 누적

    # ▶ 평균 학습 손실 계산 및 기록
    avg_train_loss = total_loss / len(train_loader)
    train_loss_history.append(avg_train_loss)
    print(f"[Train] Epoch {epoch+1}, Avg Loss = {avg_train_loss:.6f}")

    # ───────────────────────────────────────────
    # [5] 검증 루프 (Validation)
    # ───────────────────────────────────────────
    forward_net.eval()        # 평가 모드 전환
    val_loss_total = 0.0

    with torch.no_grad():    # 그래디언트 비활성화 (메모리 절약)
        for batch in val_loader:
            d_norm = batch['d_norm'].to(device)
            T_target = batch['T_target'].to(device)

            T_pred = forward_net(d_norm)           # 예측
            loss = F.mse_loss(T_pred, T_target)    # 손실 계산
            val_loss_total += loss.item()

    # ▶ 평균 검증 손실 계산 및 기록
    avg_val_loss = val_loss_total / len(val_loader)
    val_loss_history.append(avg_val_loss)
    print(f"[Val]   Epoch [{epoch+1}/{num_epochs}] done. Avg Loss: {avg_val_loss:.6f}")


[Train] Epoch 1, Avg Loss = 0.024856
[Val]   Epoch [1/600] done. Avg Loss: 0.022393
[Train] Epoch 2, Avg Loss = 0.022374
[Val]   Epoch [2/600] done. Avg Loss: 0.021987
[Train] Epoch 3, Avg Loss = 0.021553
[Val]   Epoch [3/600] done. Avg Loss: 0.020566
[Train] Epoch 4, Avg Loss = 0.020332
[Val]   Epoch [4/600] done. Avg Loss: 0.018631
[Train] Epoch 5, Avg Loss = 0.018621
[Val]   Epoch [5/600] done. Avg Loss: 0.018245
[Train] Epoch 6, Avg Loss = 0.016813
[Val]   Epoch [6/600] done. Avg Loss: 0.014914
[Train] Epoch 7, Avg Loss = 0.014887
[Val]   Epoch [7/600] done. Avg Loss: 0.013470
[Train] Epoch 8, Avg Loss = 0.013610
[Val]   Epoch [8/600] done. Avg Loss: 0.011817
[Train] Epoch 9, Avg Loss = 0.012065
[Val]   Epoch [9/600] done. Avg Loss: 0.010882
[Train] Epoch 10, Avg Loss = 0.010592
[Val]   Epoch [10/600] done. Avg Loss: 0.009454
[Train] Epoch 11, Avg Loss = 0.008899
[Val]   Epoch [11/600] done. Avg Loss: 0.008016
[Train] Epoch 12, Avg Loss = 0.007298
[Val]   Epoch [12/600] done. Avg L

KeyboardInterrupt: 

In [8]:
# ────────────────────────────────────────────────────────────────────────────
# (C) InverseNet 정의 (MLP)
# ────────────────────────────────────────────────────────────────────────────
t_path = '/content/drive/MyDrive/Colab Notebooks/Bi-directional20250402'
os.makedirs(t_path, exist_ok=True)
num_layers = len(material_sequence)  # 3
class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1198),
            nn.ReLU(),
            nn.Linear(1198, 1125),
            nn.ReLU(),
            nn.Linear(1125, 853),
            nn.ReLU(),
            nn.Linear(853, 1411),
            nn.ReLU(),
            nn.Linear(1411, 1320),
            nn.ReLU(),
            nn.Linear(1320, 1958),
            nn.ReLU(),
            nn.Linear(1958, output_dim)
        )

    def forward(self, x):
        # x must be shape (B, input_dim)
        return self.net(x)


inverse_net = InverseNet(input_dim=NUM_WAVELENGTHS, output_dim=num_layers).to(device)
inverse_net = inverse_net.to(torch.float64)
optimizer = optim.Adam(inverse_net.parameters(), lr=0.0016277391324458217)

# ────────────────────────────────────────────────────────────────────────────
# (D) 학습 루프
# ────────────────────────────────────────────────────────────────────────────
num_epochs = 600
alpha = 1.0616785666328346
forward_net.eval()
for epoch in range(num_epochs):
    inverse_net.train()
    total_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):
        T_target = batch['T_target'].to(device)      # [B, 401]
        d_norm_true = batch['d_norm'].to(device)     # [B, 3]

        optimizer.zero_grad()
        d_norm_pred = inverse_net(T_target.to(torch.float64)).squeeze(1)

        if batch_idx == 0:
            print(f"\n[DEBUG] Epoch {epoch+1}, Batch {batch_idx+1}")
        d_nm_pred = destandardize_thickness(d_norm_pred)
        d_nm_true = destandardize_thickness(d_norm_true)

        T_pred_batch = model(d_nm_pred)
        squeezed_T_target = T_target.squeeze(1)
        T_target_destd = destandardize_spectrum(squeezed_T_target)

        loss_spectrum = F.mse_loss(T_pred_batch, T_target_destd)
        loss_thickness = F.mse_loss(d_norm_pred, d_norm_true.squeeze(1))

        if epoch < 18:
            loss = loss_spectrum + alpha * loss_thickness
        else:
            loss = loss_spectrum

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"[Train] Epoch [{epoch+1}/{num_epochs}] done. Avg Loss: {avg_train_loss:.6f}")

    # ───────────────────────────────────────────
    # Validation
    # ───────────────────────────────────────────
    inverse_net.eval()
    val_loss_total = 0.0
    with torch.no_grad():
        for val_batch in val_loader:
            T_val = val_batch['T_target'].to(device)
            d_norm_val = val_batch['d_norm'].to(device)

            d_pred_val = inverse_net(T_val.to(torch.float64)).squeeze(1)
            d_nm_val_pred = destandardize_thickness(d_pred_val)
            T_val_pred = model(d_nm_val_pred)

            T_val_destd = destandardize_spectrum(T_val.squeeze(1))

            loss_val_spectrum = F.mse_loss(T_val_pred, T_val_destd)
            loss_val_thickness = F.mse_loss(d_pred_val, d_norm_val.squeeze(1))

            if epoch < 18:
                val_loss = loss_val_spectrum + alpha * loss_val_thickness
            else:
                val_loss = loss_val_spectrum

            val_loss_total += val_loss.item()

    avg_val_loss = val_loss_total / len(val_loader)
    print(f"[Val]   Epoch [{epoch+1}/{num_epochs}] done. Avg Loss: {avg_val_loss:.6f}")


[DEBUG] Epoch 1, Batch 1
[Train] Epoch [1/600] done. Avg Loss: 0.138271
[Val]   Epoch [1/600] done. Avg Loss: 0.126636

[DEBUG] Epoch 2, Batch 1
[Train] Epoch [2/600] done. Avg Loss: 0.123173
[Val]   Epoch [2/600] done. Avg Loss: 0.125824

[DEBUG] Epoch 3, Batch 1
[Train] Epoch [3/600] done. Avg Loss: 0.118725
[Val]   Epoch [3/600] done. Avg Loss: 0.107659

[DEBUG] Epoch 4, Batch 1
[Train] Epoch [4/600] done. Avg Loss: 0.106658
[Val]   Epoch [4/600] done. Avg Loss: 0.098445

[DEBUG] Epoch 5, Batch 1
[Train] Epoch [5/600] done. Avg Loss: 0.100070
[Val]   Epoch [5/600] done. Avg Loss: 0.096010

[DEBUG] Epoch 6, Batch 1
[Train] Epoch [6/600] done. Avg Loss: 0.097049
[Val]   Epoch [6/600] done. Avg Loss: 0.089660

[DEBUG] Epoch 7, Batch 1
[Train] Epoch [7/600] done. Avg Loss: 0.095435
[Val]   Epoch [7/600] done. Avg Loss: 0.092927

[DEBUG] Epoch 8, Batch 1
[Train] Epoch [8/600] done. Avg Loss: 0.089577
[Val]   Epoch [8/600] done. Avg Loss: 0.083823

[DEBUG] Epoch 9, Batch 1
[Train] Epoch 

KeyboardInterrupt: 

In [9]:
import torch
import torch.nn as nn
import pandas as pd
import urllib.request

class InverseNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1198),
            nn.ReLU(),
            nn.Linear(1198, 1125),
            nn.ReLU(),
            nn.Linear(1125, 853),
            nn.ReLU(),
            nn.Linear(853, 1411),
            nn.ReLU(),
            nn.Linear(1411, 1320),
            nn.ReLU(),
            nn.Linear(1320, 1958),
            nn.ReLU(),
            nn.Linear(1958, output_dim)
        )

    def forward(self, x):
        # x must be shape (B, input_dim)
        return self.net(x)

# ──────────────────────────────────────────────────────
# 2. 모델 불러오기 (GitHub → 로컬 저장 → 모델에 로드)
# ──────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inverse_net = InverseNet(input_dim=401, output_dim=3).to(device).to(torch.float64)

# GitHub에서 파라미터 다운로드
url = "https://raw.githubusercontent.com/KIMEUIJOON-KNU/Bi-directional/main/inverse_net_final.pth"
save_path = "inverse_net_final.pth"
urllib.request.urlretrieve(url, save_path)

# 로드
inverse_net.load_state_dict(torch.load(save_path, map_location=device))
inverse_net.eval()
print("✅ 모델 로드 및 eval() 전환 완료")

# ──────────────────────────────────────────────────────
# 3. test dataset 추출 (random split된 dataset 기반)
# ──────────────────────────────────────────────────────
from torch.utils.data import random_split, DataLoader

# 이미 정의되어 있는 dataset / train_size / val_size / test_size 사용
_, _, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ──────────────────────────────────────────────────────
# 4. test set 예측 및 결과 저장
# ──────────────────────────────────────────────────────
all_true_d, all_pred_d = [], []

with torch.no_grad():
    for batch in test_loader:
        T_test = batch['T_target'].to(device).squeeze(1)  # shape: [B, 401]
        d_true_norm = batch['d_norm'].to(device)

        # 예측
        d_pred_norm = inverse_net(T_test)

        # 정규화된 두께 → 실제 두께로 복원
        d_pred_nm = destandardize_thickness(d_pred_norm).cpu().numpy()
        d_true_nm = destandardize_thickness(d_true_norm).cpu().numpy()

        all_true_d.append(d_true_nm)
        all_pred_d.append(d_pred_nm)

# ──────────────────────────────────────────────────────
# 5. DataFrame으로 결과 정리 (d1, d2, d3 각 비교)
# ──────────────────────────────────────────────────────
true_all = torch.tensor(np.concatenate(all_true_d, axis=0))
pred_all = torch.tensor(np.concatenate(all_pred_d, axis=0))

df_result = pd.DataFrame({
    "True_d1": true_all[:, 0],
    "True_d2": true_all[:, 1],
    "True_d3": true_all[:, 2],
    "Pred_d1": pred_all[:, 0],
    "Pred_d2": pred_all[:, 1],
    "Pred_d3": pred_all[:, 2],
})

# 상위 10개 출력
print(df_result.head(10))

RuntimeError: Error(s) in loading state_dict for InverseNet:
	Missing key(s) in state_dict: "net.10.weight", "net.10.bias", "net.12.weight", "net.12.bias". 
	size mismatch for net.0.weight: copying a param with shape torch.Size([768, 401]) from checkpoint, the shape in current model is torch.Size([1198, 401]).
	size mismatch for net.0.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([1198]).
	size mismatch for net.2.weight: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([1125, 1198]).
	size mismatch for net.2.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([1125]).
	size mismatch for net.4.weight: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([853, 1125]).
	size mismatch for net.4.bias: copying a param with shape torch.Size([768]) from checkpoint